Вы получили основные навыки обработки данных, теперь пора испытать их на практике. Сейчас вам предстоит заняться задачей классификации.

Представлен датасет центра приюта животных, и вашей задачей будет обучить модель таким образом, чтобы  по определенным признакам была возможность максимально уверенно предсказать метки 'Adoption' и 'Transfer' (столбец “outcome_type”).

Здесь вы вольны делать что угодно. Я хочу видеть от вас:

1. Проверка наличия/обработка пропусков

2. Проверьте взаимосвязи между признаками

3. Попробуйте создать свои признаки

4. Удалите лишние

5. Обратите внимание на текстовые столбцы. Подумайте, что можно извлечь полезного оттуда

6. Использование профайлера вам поможет.

7. Не забывайте, что у вас есть PCA (Метод главных компонент). Он может пригодиться.

Вспомните о всем, что я говорил на предыдущих занятиях. Не все будет пригодится, но в жизни вам никто не будет говорить, что использовать :)

Хорошим классификатором для этой задачи будет "Случайный лес" (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)


Понимать суть работы "леса" не обязательно на данном этапе, но качество предсказаний будет выше, чем с линейным классификатором. (если желаете, вот гайд https://adataanalyst.com/scikit-learn/linear-classification-method/)

Желаю успеха :)

In [283]:
# load data
import pandas as pd
data = pd.read_csv('aac_shelter_outcomes.csv')
data.head(10)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male
6,1 year,A693700,Other,Squirrel Mix,Tan,2013-12-13T00:00:00,2014-12-13T12:20:00,2014-12-13T12:20:00,NaN,Suffering,Euthanasia,Unknown
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23T00:00:00,2014-12-08T15:55:00,2014-12-08T15:55:00,*Ella,Partner,Transfer,Spayed Female
8,1 month,A685067,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,2014-08-14T18:45:00,Lucy,NaN,Adoption,Intact Female
9,3 months,A678580,Cat,Domestic Shorthair Mix,White/Black,2014-03-26T00:00:00,2014-06-29T17:45:00,2014-06-29T17:45:00,*Frida,Offsite,Adoption,Spayed Female


In [284]:
data.describe()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
count,78248,78256,78256,78256,78256,78256,78256,78256,54370,35963,78244,78254
unique,46,70855,5,2128,525,5869,64361,64361,14574,19,9,5
top,1 year,A718223,Dog,Domestic Shorthair Mix,Black/White,2014-05-05T00:00:00,2016-04-18T00:00:00,2016-04-18T00:00:00,Bella,Partner,Adoption,Neutered Male
freq,14355,11,44242,23335,8153,112,39,39,344,19660,33112,27784


In [285]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78256 entries, 0 to 78255
Data columns (total 12 columns):
age_upon_outcome    78248 non-null object
animal_id           78256 non-null object
animal_type         78256 non-null object
breed               78256 non-null object
color               78256 non-null object
date_of_birth       78256 non-null object
datetime            78256 non-null object
monthyear           78256 non-null object
name                54370 non-null object
outcome_subtype     35963 non-null object
outcome_type        78244 non-null object
sex_upon_outcome    78254 non-null object
dtypes: object(12)
memory usage: 7.2+ MB


ToDo:
1. breed -> exclude Mix in other column [Contain_Mix]
2. color -> separate to first_color, second_color
3. monthyear = datetime -> Drop datetime
4. [age_upon_outcome] = round(Datediff([datetime]-[date_of_birth]))-> Drop [age_upon_outcome]; 
5. Create [age_upon_outcome_weeks] = round(Datediff([datetime]-[date_of_birth]))
6. name -> NaN is type. Usually when Euthanasia or Transfer or young Age, change to NoName Change to NoName for RandomForestClassifire
7. outcome_subtype ->NaN is type. Usually when Adoption or Return to Owner. Change to NoName for RandomForestClassifire
8. outcome_type -> dropNA (small count NULL, therefore dropNA is not reduce the quality of information significantly)
9. sex_upon_outcome -> dropNA (small count NULL, therefore dropNA is not reduce the quality of information significantly)

These activities cover the following sections of the assignment:
1. Проверка наличия/обработка пропусков
2. Проверьте взаимосвязи между признаками
3. Попробуйте создать свои признаки
4. Удалите лишние
5. Обратите внимание на текстовые столбцы. Подумайте, что можно извлечь полезного оттуда

In [286]:
# create a copy of primary data
data_backup = data.copy

In [287]:
# create function which check "Mix" and separate from string as a type
# I think that Mix is not Pure breed, therefore it can be reason outcome_type
def breed_contain_mix(row):
    mystring = row['breed']
    mystring = mystring.lower().strip()
    
    word = "mix"
    """This function check "Mix" and separate from string as a type"""
    
    if word in mystring:
#         mystring = mystring.replace('mix', '').strip()
#         row['breed'] = mystring
        return 1
    else:
        return 0


def breed_contain_mix_change_breed(row):
    mystring = row['breed']
    mystring = mystring.lower().strip()
    word = "mix"
    """This function check "Mix" and separate from string as a type"""
    if word in mystring:
        mystring = mystring.replace('mix', '').strip()
        return mystring
    else:
        return mystring    
    
# Вопрос к преподавателю 1: 
#     Как выдать из функции изменение 2х столбцов (то есть хочу чтобы  появился признак Contain_Mix и из [breed] удалилось слово Mix и пробелы)
#     И при этом не писать 2 функции, как реализовано у меня. Google'ил, но решение не нашел

In [288]:
# 1. breed -> exclude Mix in other column [Contain_Mix]
data['Contain_Mix'] =data.apply(breed_contain_mix, axis=1)
data['breed'] =data.apply(breed_contain_mix_change_breed, axis=1)
data.head(10)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,Contain_Mix
0,2 weeks,A684346,Cat,domestic shorthair,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male,1
1,1 year,A666430,Dog,beagle,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female,1
2,1 year,A675708,Dog,pit bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male,0
3,9 years,A680386,Dog,miniature schnauzer,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male,1
4,5 months,A683115,Other,bat,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown,1
5,4 months,A664462,Dog,leonberger,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male,1
6,1 year,A693700,Other,squirrel,Tan,2013-12-13T00:00:00,2014-12-13T12:20:00,2014-12-13T12:20:00,NaN,Suffering,Euthanasia,Unknown,1
7,3 years,A692618,Dog,chihuahua shorthair,Brown,2011-11-23T00:00:00,2014-12-08T15:55:00,2014-12-08T15:55:00,*Ella,Partner,Transfer,Spayed Female,1
8,1 month,A685067,Cat,domestic shorthair,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,2014-08-14T18:45:00,Lucy,NaN,Adoption,Intact Female,1
9,3 months,A678580,Cat,domestic shorthair,White/Black,2014-03-26T00:00:00,2014-06-29T17:45:00,2014-06-29T17:45:00,*Frida,Offsite,Adoption,Spayed Female,1


In [289]:
# 2. color -> separate to first_color, second_color
# new data frame with split value columns 
new = data["color"].str.split("/", n = 1, expand = True)
  
# making separate first name column from new data frame 
data["color_first"]= new[0] 
  
# making separate last name column from new data frame 
data["color_second"]= new[1] 
  
# Dropping old Name columns 
data.drop(columns =["color"], inplace = True) 
data.head(5)

,age_upon_outcome,animal_id,animal_type,breed,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,Contain_Mix,color_first,color_second
0,2 weeks,A684346,Cat,domestic shorthair,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male,1,Orange Tabby,None
1,1 year,A666430,Dog,beagle,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female,1,White,Brown
2,1 year,A675708,Dog,pit bull,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male,0,Blue,White
3,9 years,A680386,Dog,miniature schnauzer,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male,1,White,None
4,5 months,A683115,Other,bat,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown,1,Brown,None


In [290]:
# monthyear = datetime -> Drop datetime
# [age_upon_outcome] = round(Datediff([datetime]-[date_of_birth]))-> Drop [age_upon_outcome];
data.drop(['datetime', 'age_upon_outcome'], axis=1, inplace=True)
data.head(5)

,animal_id,animal_type,breed,date_of_birth,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,Contain_Mix,color_first,color_second
0,A684346,Cat,domestic shorthair,2014-07-07T00:00:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male,1,Orange Tabby,None
1,A666430,Dog,beagle,2012-11-06T00:00:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female,1,White,Brown
2,A675708,Dog,pit bull,2013-03-31T00:00:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male,0,Blue,White
3,A680386,Dog,miniature schnauzer,2005-06-02T00:00:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male,1,White,None
4,A683115,Other,bat,2014-01-07T00:00:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown,1,Brown,None


In [291]:
# Create [age_upon_outcome_weeks] = round(Datediff([datetime]-[date_of_birth]))
import datetime

#Convert to DateTime
data['monthyear'] = pd.to_datetime(data['monthyear'])
data['date_of_birth'] = pd.to_datetime(data['date_of_birth'])

# Create new Features
data['age_upon_outcome_weeks'] = data['monthyear']-data['date_of_birth']
data['age_upon_outcome_weeks'] = data['age_upon_outcome_weeks']/np.timedelta64(1, 'W')
data['age_upon_outcome_days'] = (data['monthyear']-data['date_of_birth']) /np.timedelta64(1, 'D')
data['month_outcome'] = pd.DatetimeIndex(data['monthyear']).month
data['year_outcome'] = pd.DatetimeIndex(data['monthyear']).year
data['weekday_outcomes'] = pd.DatetimeIndex(data['monthyear']).dayofweek

data.drop(['date_of_birth', 'monthyear'], axis=1, inplace=True)

data.head(5)

,animal_id,animal_type,breed,name,outcome_subtype,outcome_type,sex_upon_outcome,Contain_Mix,color_first,color_second,age_upon_outcome_weeks,age_upon_outcome_days,month_outcome,year_outcome,weekday_outcomes
0,A684346,Cat,domestic shorthair,NaN,Partner,Transfer,Intact Male,1,Orange Tabby,None,2.238492,15.669444,7,2014,1
1,A666430,Dog,beagle,Lucy,Partner,Transfer,Spayed Female,1,White,Brown,52.355853,366.490972,11,2013,3
2,A675708,Dog,pit bull,*Johnny,NaN,Adoption,Neutered Male,0,Blue,White,61.371032,429.597222,6,2014,1
3,A680386,Dog,miniature schnauzer,Monday,Partner,Transfer,Neutered Male,1,White,None,471.522817,3300.659722,6,2014,6
4,A683115,Other,bat,NaN,Rabies Risk,Euthanasia,Unknown,1,Brown,None,25.940873,181.586111,7,2014,0


In [292]:
# name -> NaN is type. Usually when Euthanasia or Transfer or young Age, change to NoName Change to NoName for RandomForestClassifire
# outcome_subtype ->NaN is type. Usually when Adoption or Return to Owner. Change to NoName for RandomForestClassifire
data[['name', 'outcome_subtype', 'color_second']] = data[['name', 'outcome_subtype', 'color_second']].fillna('NoName')
data.head(5)

,animal_id,animal_type,breed,name,outcome_subtype,outcome_type,sex_upon_outcome,Contain_Mix,color_first,color_second,age_upon_outcome_weeks,age_upon_outcome_days,month_outcome,year_outcome,weekday_outcomes
0,A684346,Cat,domestic shorthair,NoName,Partner,Transfer,Intact Male,1,Orange Tabby,NoName,2.238492,15.669444,7,2014,1
1,A666430,Dog,beagle,Lucy,Partner,Transfer,Spayed Female,1,White,Brown,52.355853,366.490972,11,2013,3
2,A675708,Dog,pit bull,*Johnny,NoName,Adoption,Neutered Male,0,Blue,White,61.371032,429.597222,6,2014,1
3,A680386,Dog,miniature schnauzer,Monday,Partner,Transfer,Neutered Male,1,White,NoName,471.522817,3300.659722,6,2014,6
4,A683115,Other,bat,NoName,Rabies Risk,Euthanasia,Unknown,1,Brown,NoName,25.940873,181.586111,7,2014,0


In [293]:
# outcome_type -> dropNA (small count NULL, therefore dropNA is not reduce the quality of information significantly)
# sex_upon_outcome -> dropNA (small count NULL, therefore dropNA is not reduce the quality of information significantly)
data.dropna(inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78243 entries, 0 to 78255
Data columns (total 15 columns):
animal_id                 78243 non-null object
animal_type               78243 non-null object
breed                     78243 non-null object
name                      78243 non-null object
outcome_subtype           78243 non-null object
outcome_type              78243 non-null object
sex_upon_outcome          78243 non-null object
Contain_Mix               78243 non-null int64
color_first               78243 non-null object
color_second              78243 non-null object
age_upon_outcome_weeks    78243 non-null float64
age_upon_outcome_days     78243 non-null float64
month_outcome             78243 non-null int64
year_outcome              78243 non-null int64
weekday_outcomes          78243 non-null int64
dtypes: float64(2), int64(4), object(9)
memory usage: 9.6+ MB


In [294]:
#Create profileReport
import pandas as pd
import numpy as np
import pandas_profiling
from pandas_profiling import ProfileReport
pandas_profiling.ProfileReport(data)

# Conclusion: ProfileReport show 10% dublicates. Drop them

C:\Users\User\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,16
Number of observations,78243
Total Missing (%),0.0%
Total size in memory,9.6 MiB
Average record size in memory,128.0 B
Numeric,5
Categorical,9
Boolean,1
Date,0
Text (Unique),0
Rejected,1


In [295]:
data = data.drop_duplicates()

# [animal_id] has duplicates. 
# If the animal was once returned to the shelter, the likelihood of a re-return is likely to increase. 
# Therefore, we calculate the number of encountered count_animal_id and delete the animal_id field
animal_id_count = pd.DataFrame(data[['animal_id', 'animal_type']].groupby(['animal_id']).count())
animal_id_count.reset_index(inplace = True)
animal_id_count.rename(columns={"animal_type": "count_of_animal_id"}, inplace = True)


animal_id_count.head()


,animal_id,count_of_animal_id
0,A006100,3
1,A047759,1
2,A134067,1
3,A141142,1
4,A163459,1


In [296]:
data = pd.merge(data, animal_id_count, how='left', on=['animal_id'])

# Drop animal_id becuse it is not contain information. For help model
data.drop(['animal_id'], axis=1, inplace=True)
data.head(10)
# df.merge(pd.DataFrame(data = [s.values] * len(s), columns = s.index), left_index=True, right_index=True)


,animal_type,breed,name,outcome_subtype,outcome_type,sex_upon_outcome,Contain_Mix,color_first,color_second,age_upon_outcome_weeks,age_upon_outcome_days,month_outcome,year_outcome,weekday_outcomes,count_of_animal_id
0,Cat,domestic shorthair,NoName,Partner,Transfer,Intact Male,1,Orange Tabby,NoName,2.238492,15.669444,7,2014,1,1
1,Dog,beagle,Lucy,Partner,Transfer,Spayed Female,1,White,Brown,52.355853,366.490972,11,2013,3,1
2,Dog,pit bull,*Johnny,NoName,Adoption,Neutered Male,0,Blue,White,61.371032,429.597222,6,2014,1,1
3,Dog,miniature schnauzer,Monday,Partner,Transfer,Neutered Male,1,White,NoName,471.522817,3300.659722,6,2014,6,1
4,Other,bat,NoName,Rabies Risk,Euthanasia,Unknown,1,Brown,NoName,25.940873,181.586111,7,2014,0,1
5,Dog,leonberger,*Edgar,Partner,Transfer,Intact Male,1,Brown,White,18.077976,126.545833,10,2013,0,1
6,Other,squirrel,NoName,Suffering,Euthanasia,Unknown,1,Tan,NoName,52.216270,365.513889,12,2014,5,1
7,Dog,chihuahua shorthair,*Ella,Partner,Transfer,Spayed Female,1,Brown,NoName,158.809028,1111.663194,12,2014,0,1
8,Cat,domestic shorthair,Lucy,NoName,Adoption,Intact Female,1,Blue Tabby,White,8.540179,59.781250,8,2014,3,1
9,Cat,domestic shorthair,*Frida,Offsite,Adoption,Spayed Female,1,White,Black,13.677083,95.739583,6,2014,6,1


In [299]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# RandomForestClassifier don't work with a strings, therefore I will use LabelEncoder
string_cols_to_labele = ('animal_type', 
                         'breed', 
                         'name', 
                         'outcome_subtype', 
                         'outcome_type', 
                         'sex_upon_outcome', 
                         'color_first', 
                         'color_second')

for i in string_cols_to_labele:
    data[i] = le.fit_transform(data[i])

data.head()
pandas_profiling.ProfileReport(data)


# data is prepared to Create Model
# I will use RandomForestClassifier

C:\Users\User\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,15
Number of observations,78233
Total Missing (%),0.0%
Total size in memory,9.5 MiB
Average record size in memory,128.0 B
Numeric,13
Categorical,0
Boolean,1
Date,0
Text (Unique),0
Rejected,1


In [298]:
X = data.loc[:, data.columns != 'outcome_type']
y = data['outcome_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
forest = RandomForestClassifier(n_estimators=5, random_state=2)
forest.fit(X_train, y_train)
print("Правильность на тестовом наборе: {:.3f}".format(forest.score(X_test, y_test)))


Правильность на тестовом наборе: 0.889


In [282]:
from sklearn.metrics import classification_report

y_pred = forest.predict(X_test)

print(classification_report(y_test, y_pred))

# Conclusions:
# #     Depending on the business task, we will focus on Precision or Recall.
# We will focus on Precision when the cost of each study is important (High) 
# (it should not fall into the study of the wrong elements), 
# and we will focus on Recall when it is important not to miss a sick patient, 
# such as a cancer patient

# Our choice depends on the needs of business users and is beyond the scope of this homework.

              precision    recall  f1-score   support

           0       0.86      0.89      0.88      8277
           1       0.77      0.64      0.69       170
           2       0.67      0.69      0.68        77
           3       0.98      0.94      0.96      1520
           4       1.00      0.09      0.17        11
           5       0.00      0.00      0.00         4
           6       0.74      0.70      0.72      3588
           7       0.00      0.00      0.00        38
           8       0.99      1.00      1.00      5874

    accuracy                           0.89     19559
   macro avg       0.67      0.55      0.57     19559
weighted avg       0.89      0.89      0.89     19559



C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
